1. Data preparation
2. Create additional features
3. Algorithm creation
4. Test algorithm
5. Create test predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. Data preparation

First lets import the data and generate one set to process all data at once

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train['set'], test['set'] = 'train', 'test'
combined = pd.concat([train, test])

### Missing values
#### First step is to fill missing values and drop columns we will not be using.

In [ ]:
combined.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
set               0
dtype: int64

#### Only one Fare price is missing. We shall fill it with the median of Pclass:

In [ ]:
pclass = combined.loc[combined.Fare.isnull(), 'Pclass'].values[0]
median_fare = combined.loc[combined.Pclass== pclass, 'Fare'].median()
combined.loc[combined.Fare.isnull(), 'Fare'] = median_fare

#### Missing ages
To fill in the missing ages, we can do something more clever then just take the overal median age. The names contain titles of which some are linked to their age. Master is a younger boy (in general). Lets take the median of each age group.

In [ ]:
# Select everything before the . as title
combined['Title'] = combined['Name'].str.extract('([A-Za-z]+)\.', expand=True)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [ ]:
title_reduction = {'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 
                   'Master': 'Master', 'Don': 'Mr', 'Rev': 'Rev',
                   'Dr': 'Dr', 'Mme': 'Miss', 'Ms': 'Miss',
                   'Major': 'Mr', 'Lady': 'Mrs', 'Sir': 'Mr',
                   'Mlle': 'Miss', 'Col': 'Mr', 'Capt': 'Mr',
                   'Countess': 'Mrs','Jonkheer': 'Mr',
                   'Dona': 'Mrs'}
combined['Title'] = combined['Title'].map(title_reduction)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rev', 'Dr'], dtype=object)

In [ ]:
for title, age in combined.groupby('Title')['Age'].median().iteritems():
    print(title, age)
    combined.loc[(combined['Title']==title) & (combined['Age'].isnull()), 'Age'] = age

Dr 49.0
Master 4.0
Miss 22.0
Mr 30.0
Mrs 36.0
Rev 41.5


In [ ]:
combined.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
set               0
Title             0
dtype: int64

# 2. Create additional features

We can note that there could be groups of people, who would help each other. This might affect the survivability.

In [ ]:
def other_family_members_survived(dataset, label='family_survival'):
    """
    Check if other family members survived
      -> 0 other did not survive
      -> 1 at least one other family member survived
      -> 0.5 unknown if other members survived or person was alone
    
    Parameters
    ----------
    dataset : DataFrame
      The sub-dataframe containing the family
    """
    ds = dataset.copy()
    if len(dataset) == 1:
        ds[label] = 0.5
        return ds
    result = []
    for ix, row in dataset.iterrows():
        survived_fraction = dataset.drop(ix)['Survived'].mean()
        if np.isnan(survived_fraction):
            result.append(0.5)
        elif survived_fraction == 0:
            result.append(0)
        else:
            result.append(1)
    ds[label] = result
    return ds

In [ ]:
combined['surname'] = combined['Name'].apply(lambda x: x.split(",")[0])
combined = combined.groupby(['surname', 'Fare']).apply(other_family_members_survived).reset_index(drop=True)

Missing data on families can also be extracted from Tickets. Same ticket orders have the same ticket number.

In [ ]:
combined = combined.groupby(['Ticket']).apply(lambda x: other_family_members_survived(x, label='family_survival_ticket')).reset_index(drop=True)
combined.loc[combined['family_survival'] == 0.5, 'family_survival'] = combined.loc[combined['family_survival'] == 0.5, 'family_survival_ticket']

We get family size from Parch and Sibsp

In [ ]:
combined['family_size'] = combined['Parch'] + combined['SibSp']

KNN needs numerical features therefore, we will convert them to numbers. In a general sense, binary categorical data can work. For larger categorical groups, it only makes sense when the numerical values itself have meaning. For example, for class levels, the difference between first class and third class actually mean something. On the other hand, if we would convert Embarked to a number, there is no meaning in the difference between embarked 1 and embarked 2.

In [ ]:
combined['Sex'] = LabelEncoder().fit_transform(combined['Sex'])

In [ ]:
combined.loc[:, 'Age'] = pd.qcut(combined['Age'], 4, labels=False)
combined.loc[:, 'Fare'] = pd.qcut(combined['Fare'], 5, labels=False)

#### Select only coluns we will use and then we scale them

1.   List item
2.   List item



In [ ]:
selected = ['Pclass', 'Sex', 'Age', 'Fare', 'family_size', 'family_survival']
scaler  = StandardScaler()
scaler.fit(combined[selected])
combined[selected] = scaler.transform(combined[selected])

In [ ]:
combined.to_parquet('titanic_family_survivabillity.parquet', index=False)

#### Now we split the sets back to train/test

In [ ]:
train = combined.loc[combined['set'] == 'train'].drop('set', axis=1).reset_index(drop=True)
test = combined.loc[combined['set'] == 'test'].drop(['set', 'Survived'], axis=1).reset_index(drop=True)

# 3. Algorithm
KNN works by finding, as the name suggests, the nearest neighbor. It assumes that classes share similar properties. To compare to points, we have to see them as vectors. Each feature adds to a dimension. Would we only have a single feature it would be the numeric distance between the two points. With two or more features, we need to do something smarter. To create a single number to express the similarity (or distance) we need to agegrate all the features (or dimensions). One way is to use the Euclidean distance, which is the root of the sum of squared distances between all features. There are many other ways to agegrate the feature distances, all with their strengths and weaknesses. For this, we will use the Euclidean distance.

In [ ]:
def euclidean_distance(vector1, vector2):
    return np.sqrt(np.sum((vector1 - vector2)**2))

# test function
vec1 = np.array([3, 0])
vec2 = np.array([0, 4])

# this is the 3:4:5 triangle and therefore, it should return 5 (Long live Pythagoras)
euclidean_distance(vec1, vec2)

5.0

Using our distance function we will now find the closest match in our dataset, when providing a vector.

In [ ]:
# A first implementation
def get_nearest_neighbor(vector, dataset, number_of_neighbors=1, ignore_cols=['Survived']):
    distances = []
    for ix, row in dataset.loc[:, ~dataset.columns.isin(ignore_cols)].iterrows():
        distance = euclidean_distance(row, vector)
        distances.append((distance, ix))
    indices = [x[1] for x in sorted(distances, key=lambda x: x[0])]
    neighbors = dataset.loc[indices[:number_of_neighbors]]
    return neighbors

# Another implementation using Pandas
def get_nearest_neighbor(vector, dataset, number_of_vectors=1, ignore_cols=['Survived'], not_count_duplicates=False):
    ds = dataset.copy()
    ds['distance'] = ds.loc[:, ~ds.columns.isin(ignore_cols)].apply(
        lambda x: euclidean_distance(x, vector), axis=1)
    if not_count_duplicates:
        distances = sorted(ds.distance.unique())[:number_of_vectors]
        return ds.loc[ds.distance <= max(distances)].drop('distance', axis=1)
    return ds.sort_values('distance', ascending=True).head(number_of_vectors).drop('distance', axis=1)
        
# test function
dataset = pd.DataFrame([
    {'a': 1, 'b': 1, 'Survived': 1},
    {'a': 2, 'b': 2, 'Survived': 1},
    {'a': 3, 'b': 3, 'Survived': 0},
    {'a': 4, 'b': 4, 'Survived': 0},
    {'a': 5, 'b': 5, 'Survived': 0},
])
vector = pd.Series({'a': 2.5, 'b': 2.5})

# should be (2,2) and (3,3) (if keeping track of duplicates)
get_nearest_neighbor(vector, dataset)

,a,b,Survived
1,2,2,1


Now that we have a function to list the nearest neighbors, we need to select the most dominant class of those neighbors to select as our prediction.

In [ ]:
def predict(vector, dataset, number_of_neighbors=1, y='Survived'):
    neighbors = get_nearest_neighbor(vector, dataset, number_of_neighbors)
    return round(neighbors[y].mean())

# test function
print(predict(vector, dataset))
print(predict(pd.Series({'a': 4.5, 'b': 4.5}), dataset))

1
0


# 4. Test algorithm on Titanic dataset

To test, we select on row from the set and use KNN to find the best candidate. Of course, we will remove that row from the dataset, or we would find a distance of zero as the identical row is in there.

In [ ]:
def predict_dataset(dataset, number_of_neighbors=1):
    ds = dataset.copy()
    def predict_row(vector, dataset):
        subset = dataset.loc[~(dataset.index==vector.name)]
        if vector.name % 100 == 0:
            print(vector.name)
        return int(predict(vector, subset, number_of_neighbors))

    ds['predicted'] = ds.loc[:, ds.columns.isin(selected)].apply(
        lambda x: predict_row(x, ds), axis=1)
    
    return ds

ds = predict_dataset(train, number_of_neighbors=10)

print('Accuracy:', sum(ds['Survived'] == ds['predicted']) / len(ds))


0
100
200
300
400
500
600
700
800
Accuracy: 0.835016835016835


We find a accuracy of 83.5% on the training set. Lets now make our test set predictions.

## 5. create prediction for testset

In [ ]:
def predict_testset(test_dataset, train_dataset, number_of_neighbors=1):
    ds = test_dataset.copy()
    select = selected + ['Survived']
    
    def predict_row(vector, dataset):
        if vector.name % 100 == 0:
            print(vector.name)
        return int(predict(vector, dataset[select], number_of_neighbors))

    ds['Survived'] = ds.loc[:, ds.columns.isin(selected)].apply(
        lambda x: predict_row(x, train_dataset), axis=1)
    
    return ds

In [ ]:
final_test = predict_testset(test, train, number_of_neighbors=10)

0
100
200
300
400


In [ ]:
result = final_test[['PassengerId', 'Survived']].copy()

In [ ]:
result

,PassengerId,Survived
0,1227,0
1,1050,0
2,1128,0
3,1083,0
4,1158,0
...,...,...
413,1114,1
414,925,0
415,1136,0
416,1059,0
